In [77]:
%load_ext autoreload
%autoreload 2

from space_sim.plot.plot import plot_position, plot_velocity, plot_acceleration, plot_3d_position, wrap_plots_in_system

import ipywidgets as widgets
from ipywidgets import interactive_output

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
a_knob = widgets.BoundedFloatText(value=1, min=0.1, max=15.0, step=0.1, description='Acceleration (m.s-2)')
v0x_knob = widgets.IntText(value=0, step=10, description='v0.x (m.s-1)')
v0y_knob = widgets.IntText(value=0, step=10, description='v0.y (m.s-1)')
v0z_knob = widgets.IntText(value=0, step=10, description='v0.z (m.s-1)')
vTx_knob = widgets.IntText(value=0, step=10, description='vT.x (m.s-1)')
vTy_knob = widgets.IntText(value=0, step=10, description='vT.y (m.s-1)')
vTz_knob = widgets.IntText(value=0, step=10, description='vT.z (m.s-1)')

p0x_knob = widgets.IntText(value=0, step=10, description='p0.x (m)')
p0y_knob = widgets.IntText(value=0, step=10, description='p0.y (m)')
p0z_knob = widgets.IntText(value=0, step=10, description='p0.z (m)')

pTx_knob = widgets.IntText(value=100000, step=10, description='pT.x (m)')
pTy_knob = widgets.IntText(value=0, step=10, description='pT.y (m)')
pTz_knob = widgets.IntText(value=0, step=10, description='pT.z (m)')

ui = widgets.VBox([
    widgets.HBox([a_knob]),
    widgets.HBox([v0x_knob, v0y_knob, v0z_knob]),
    widgets.HBox([vTx_knob, vTy_knob, vTz_knob]),
    widgets.HBox([p0x_knob, p0y_knob, p0z_knob]),
    widgets.HBox([pTx_knob, pTy_knob, pTz_knob])
])
display(ui)

In [ ]:
from space_sim.maths.helpers import (
    acceleration_algorithm,
    acceleration_algorithm2,
    align_x_to_vector_dcm,
    make_vector,
    motion_params,
    solve_system,
    eval_in_T,
    make_equation_matrix,
    make_equation_matrix2,
    solve_RT_1d,
    Piecewise
)
from space_sim.maths.workflow import (
    initialize_system,
    initialize_unknowns,
    create_body_frame,
)
from sympy import symbols, Matrix, solveset, nonlinsolve, Max, solve, S, Eq
from sympy.physics.vector import ReferenceFrame

t = symbols("t", real=True, nonnegative=True)

A = ReferenceFrame("A")
p0x = symbols("p_{0x}")
pTx = symbols("p_{Tx}")
v0x, v0y, v0z = symbols("v_{0x} v_{0y} v_{0z}")
vTx, vTy, vTz = symbols("v_{Tx} v_{Ty} v_{Tz}")

p0 = make_vector(A, p0x, 0, 0)
pT = make_vector(A, pTx, 0, 0)

v0 = make_vector(A, v0x, v0y, v0z)
vT = make_vector(A, vTx, vTy, vTz)

direction = (pT - p0).express(A)
# v0_A = v0.express(A)
# vT_A = vT.express(A)

unknowns = initialize_unknowns()

M = symbols("M", nonnegative=True)
a_vector = acceleration_algorithm(A, **unknowns)
# a_vector = acceleration_algorithm2(A, M, **unknowns)

a_fn, v_fn, p_fn = motion_params(A, a_vector, v0, t)

computed_vT = eval_in_T(v_fn, t, unknowns["T"], unknowns["Rx"], unknowns["Ry"], unknowns["Rz"])
computed_pT = eval_in_T(p_fn, t, unknowns["T"], unknowns["Rx"], unknowns["Ry"], unknowns["Rz"])
computed_M = (unknowns["Mx"] * A.x + unknowns["My"] * A.y + unknowns["Mz"] * A.z).magnitude()

solved_Rx, solved_Tx = solve_RT_1d(A.x, computed_pT, computed_vT, pT, vT, unknowns["Rx"], unknowns["T"])
solved_Ry, solved_Ty = solve_RT_1d(A.y, computed_pT, computed_vT, pT, vT, unknowns["Ry"], unknowns["T"])
solved_Rz, solved_Tz = solve_RT_1d(A.z, computed_pT, computed_vT, pT, vT, unknowns["Rz"], unknowns["T"])

# FIXME: solved_My and solved_Mz are not defined in 0.
# They should be defined, related to the case where v0y = vTy = 0 (resp. v0z = vTZ = 0)
solved_My = solveset(solved_Tx - solved_Ty, unknowns["My"])
solved_Mz = solveset(solved_Tx - solved_Tz, unknowns["Mz"])
solved_Mx = solveset(computed_M.subs({unknowns["My"]: solved_My, unknowns["Mz"]: solved_Mz}) - M, unknowns["Mx"])

solved_T = solved_Tx.subs({unknowns["Mx"]: solved_Mx})

solved_My_final = solved_Mx.subs({unknown["Mx"]: solved_Mx})
solved_Mz_final = solved_Mz.subs({unknown["Mx"]: solved_Mx})

solved_Rx_final = solved_Rx.subs({unknowns["Mx"]: solved_Mx})
solved_Ry_final = solved_Ry.subs({unknowns["My"]: solved_My_final})
solved_Rz_final = solved_Rz.subs({unknowns["Mz"]: solved_Mz_final})

In [169]:
Ty = list(solveset((computed_pT - pT).dot(A.y).subs({unknowns["Ry"]: solved_Ry}), unknowns["T"]))[0]
tmp_Ty = Piecewise((Ty, unknowns["My"] > 0), (0, True))
tmp_My = solveset(solved_Tx - tmp_Ty, unknowns["My"], domain=S.Reals)

Tz = list(solveset((computed_pT - pT).dot(A.z).subs({unknowns["Rz"]: solved_Rz}), unknowns["T"]))[0]
tmp_Tz = Piecewise((Tz, unknowns["Mz"] > 0), (0, True))
tmp_Mz = solveset(solved_Tx - tmp_Tz, unknowns["Mz"], domain=S.Reals)

tmp_Mz
# solved_Mx = solveset(computed_M.subs({unknowns["My"]: tmp_My, unknowns["Mz"]: tmp_Mz}) - M, unknowns["Mx"])

Intersection(FiniteSet(M_x*(v_{0z} + v_{Tz} - sqrt(2*v_{0z}**2 + 2*v_{Tz}**2))/(v_{0x} + v_{Tx} + sqrt(2)*sqrt(2*G*M_x*p_{Tx} + v_{0x}**2 + v_{Tx}**2))), Interval.open(0, oo))

In [ ]:
knob_args = {
    "a_magnitude": a_knob,
    "v0x": v0x_knob, "v0y": v0y_knob, "v0z": v0z_knob,
    "vTx": vTx_knob, "vTy": vTy_knob, "vTz": vTz_knob,
    "p0x": p0x_knob, "p0y": p0y_knob, "p0z": p0z_knob,
    "pTx": pTx_knob, "pTy": pTy_knob, "pTz": pTz_knob
}

plot_p, plot_v, plot_a, plot_3d = wrap_plots_in_system(plot_position, plot_velocity, plot_acceleration, plot_3d_position)

In [ ]:
position_plot = interactive_output(plot_p, knob_args)
velocity_plot = interactive_output(plot_v, knob_args)
acceleration_plot = interactive_output(plot_a, knob_args)
position_3d_plot = interactive_output(plot_3d, knob_args)
display(position_plot, velocity_plot, acceleration_plot, position_3d_plot)